In [5]:
import numpy as np
import pandas as pd
import _pickle as pickle
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA

In [2]:
train = pd.read_csv('./data/data_clean1.csv')

In [3]:
# present some info
train.head(5)
train.info()
train.shape

,target,0,2,3,5,6,7,10,11,12,...,pc_corr_643,pc_corr_644,pc_corr_645,pc_corr_646,pc_corr_647,pc_corr_648,pc_corr_649,pc_corr_650,pc_corr_651,pc_corr_652
0,38000000.0,0.0,0.0,0,0,0,0,0.0,0,0,...,-0.001285,-0.000946,-0.007095,-0.003392,-0.008646,-0.016192,-0.000664,-0.003713,-0.000427,-0.000673
1,600000.0,0.0,0.0,0,0,0,0,0.0,0,0,...,-0.001285,-0.000946,-0.007095,-0.003392,-0.008646,-0.016192,-0.000664,-0.003713,-0.000427,-0.000673
2,10000000.0,0.0,0.0,0,0,0,0,0.0,0,0,...,-0.001285,-0.000946,-0.007095,-0.003392,-0.008646,-0.016192,-0.000664,-0.003713,-0.000427,-0.000673
3,2000000.0,0.0,0.0,0,0,0,0,0.0,0,0,...,-0.001285,-0.000946,-0.007095,-0.003392,-0.008646,-0.016192,-0.000664,-0.003713,-0.000427,-0.000673
4,14400000.0,0.0,0.0,0,0,0,0,0.0,0,0,...,-0.001285,-0.000946,-0.007095,-0.003392,-0.008646,-0.016192,-0.000664,-0.003713,-0.000427,-0.000673


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4459 entries, 0 to 4458
Columns: 4076 entries, target to pc_corr_652
dtypes: float64(1931), int64(2145)
memory usage: 138.7 MB


(4459, 4076)

In [8]:
# if a certain value takes more than 99% of the data then 
#  ( in my opinion) it doesn't contain much useful information,

n_rows = train.shape[0]
h_sk_cols = [] # highly skewed columns
for col in train.columns :
    val_counts = train[col].value_counts().tolist()
    if val_counts[0] / n_rows >= .9 :
        h_sk_cols.append(col)
#-------------------------------
print('highly skewed columns : ', len(h_sk_cols))
#-------------------------------
# there are 4615 skewed columns which is alot,
# instead of just deleting them, I'll use PCA dimension reduction to reduce them to 4 PCs.
#> first before doing that we need to make sure that the data is centered around 0 mean.
skewed_data = train[h_sk_cols].copy()
scaler = MinMaxScaler()
skewed_data = scaler.fit_transform(skewed_data)

pca = PCA(n_components=4)
new_data = pca.fit_transform(skewed_data)
print(pca.explained_variance_ratio_)
#--------------------------------


highly skewed columns :  3683
[0.04594182 0.04335799 0.03602263 0.03467276]


In [12]:
# convert the array into dataframe, set names to the columns
columns = ['pc_skew_%d' % i for i in range(new_data.shape[1])]
PCs_df = pd.DataFrame(data=new_data, columns=columns)
# drop the columns we used in PCA and then add the 2 dfs together
train.drop(columns=h_sk_cols, inplace=True)
train = pd.concat((train, PCs_df), axis=1)
# present some info
train.head(5)
train.info()
train.shape

,target,13,29,31,37,38,108,125,155,207,...,pc_corr_633,pc_corr_636,pc_corr_641,pc_corr_646,pc_corr_647,pc_corr_648,pc_skew_0,pc_skew_1,pc_skew_2,pc_skew_3
0,38000000.0,0,0.0,0.0,1300000.0,0.0,4333333.34,0,0.0,0.0,...,-0.018576,-0.003749,-0.003416,-0.003392,-0.008646,-0.016192,-0.045541,-0.074972,-0.046423,0.068788
1,600000.0,0,0.0,0.0,0.0,0.0,2000000.00,0,0.0,0.0,...,-0.018576,-0.003749,-0.003416,-0.003392,-0.008646,-0.016192,-0.023795,-0.067308,-0.039767,0.077490
2,10000000.0,0,0.0,0.0,0.0,0.0,0.00,0,0.0,0.0,...,-0.018576,-0.003749,-0.003416,-0.003392,-0.008646,-0.016192,-0.058563,-0.077023,-0.061184,0.069924
3,2000000.0,0,0.0,0.0,0.0,0.0,0.00,0,0.0,0.0,...,-0.018576,-0.003749,-0.003416,-0.003392,-0.008646,-0.016192,-0.057111,-0.084460,-0.057826,0.073179
4,14400000.0,0,0.0,0.0,0.0,0.0,0.00,0,0.0,0.0,...,-0.018576,-0.003749,-0.003416,-0.003392,-0.008646,-0.016192,-0.056772,-0.085678,-0.057723,0.073176


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4459 entries, 0 to 4458
Columns: 397 entries, target to pc_skew_3
dtypes: float64(374), int64(23)
memory usage: 13.5 MB


In [13]:
# perfect we finished :D
# save to disk
train.to_csv('./data/data_clean2.csv',index=False)